In [1]:
!kaggle competitions download -c elec-378-sp-2025-audio-emotion-classification

401 - Unauthorized


In [2]:
print('h')

h


In [ ]:
import librosa
import librosa.display
import numpy as np
import torch
import matplotlib.pyplot as plt

PATH_TO_TRAIN = 

# Load the audio file
y, sr = librosa.load("your_audio.wav", sr=None)  # sr=None keeps original sample rate


In [ ]:
import librosa.display
import matplotlib.pyplot as plt

# Compute mel spectrogram
S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)       #Converts Audio to Spectrogram
S_dB = librosa.power_to_db(S, ref=np.max)   #To decivels

# Optional: visualize it
plt.figure(figsize=(10, 4))
librosa.display.specshow(S_dB, sr=sr, x_axis='time', y_axis='mel')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel spectrogram')
plt.tight_layout()
plt.show()

In [ ]:
# Extract MFCC features
# mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

# # Optional: visualize it
# plt.figure(figsize=(10, 4))
# librosa.display.specshow(mfccs, x_axis='time', sr=sr)
# plt.colorbar()
# plt.title('MFCC')
# plt.tight_layout()
# plt.show()

# Both Mel spectrograms and MFCCs return a 2D NumPy array:
# Shape: (n_features, time_frames)

In [ ]:
# Step 3: Convert to PyTorch Tensor (CNN expects 3D: [channels, height, width])
mel_tensor = torch.tensor(S_dB).unsqueeze(0)  # shape: [1, 128, T]                      
print("Mel Tensor Shape:", mel_tensor.shape)

In [ ]:
batch = torch.stack([mel_tensor, mel_tensor, ...])  # shape: [B, 1, 128, T]



In [ ]:
class AudioCNN(nn.Module):
    def __init__(self, num_classes=7):
        super(AudioCNN, self).__init__()

        self.conv_block1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),  # input: [B, 1, 128, T]
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2))  # output: [B, 16, 64, T/2]
        )

        self.conv_block2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2))  # output: [B, 32, 32, T/4]
        )

        self.conv_block3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AdaptiveMaxPool2d((4, 4))  # output: [B, 64, 4, 4] — fixed size!
        )

        self.fc = nn.Sequential(
            nn.Flatten(),                # [B, 64*4*4]
            nn.Linear(64 * 4 * 4, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.fc(x)
        return x

# Example Usage of the CNN
# Notes
# You may need to pad or crop your input spectrograms to a consistent time dimension (e.g., 128x173).

# If you have varying lengths, consider using torch.nn.AdaptiveAvgPool2d or padding logic in a custom Dataset.

#  Next Steps:
# Want help building the Dataset and DataLoader to feed .wav files through this pipeline?
# Or want to include MFCC or augmentation options?

In [ ]:

# Simulate a single input: [batch, channels, height, width]
dummy_input = torch.randn(1, 1, 128, 173)  # 128 Mel bands, ~173 time frames
model = AudioCNN(num_classes=7)
output = model(dummy_input)

print("Output shape:", output.shape)  # [1, 7]
